In [1]:
import sys
sys.path.append('../')
from utils import load_openapi_spec, analyze_openapi_spec
import json
import yaml
from yaml import Loader
from agent.convert_request import type_check

In [2]:
api_path = 'data/eval_real.json'

In [3]:
api_data = json.load(open(api_path, "r"))

In [4]:
api = api_data[0]
api["Instances"] = []
openapi_spec = load_openapi_spec(api["Documentation"])
input_valid, output_valid = analyze_openapi_spec(openapi_spec)

Нужно попробовать сформировать запрос к api

In [5]:
from agent.convert_request import call_api_function
from utils import load_openapi_spec, escape
from agent.tools import Tool, GetDetailsTool, tool_projection

In [6]:
api_data = json.load(open('../data/eval_simulated.json', "r"))
server_url = "http://127.0.0.1:5678"
api_data = api_data[0]

In [7]:
openapi_spec = load_openapi_spec(api_data["Documentation"], replace_refs=True)
components_descriptions = escape(api_data["Function_Description"]["components"])
tools = dict()
for idx, func_name in enumerate(api_data["Function_Projection"]):
    description = escape(api_data["Function_Description"][func_name])
    if idx == len(api_data["Function_Projection"]) - 1:
        description += components_descriptions
    path, method = api_data["Function_Projection"][func_name]
    tools[func_name] = Tool(
        base_url=server_url + "/" + api_data["Name"] if server_url else None,
        func_name=func_name,
        openapi_spec=openapi_spec,
        path=path,
        method=method,
        description=description,
        retrieval_available="retrieval" in api_data.get("external_tools", [])
    )

In [8]:
tools

{'getRandomAxolotlImage': Tool(name='getRandomAxolotlImage', description='Retrieve a random axolotl image with information on the image source.\nParameters: {{}}\nOutput: Successful response.\n - Format: application/json\n - Structure: Object{{url, source, description}}', args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fda8791f610>, func=<function Tool.__init__.<locals>.func at 0x7fda89757d90>, coroutine=None, max_output_len=3000),
 'searchAxolotlImages': Tool(name='searchAxolotlImages', description='Search for axolotl images based on specific criteria such as color, gender, and size.\nParameters: {{"color": "string. One of: [wild, leucistic, albino]. The color of the axolotl (e.g., \'wild\', \'leucistic\', \'albino\', etc.).", "gender": "string. One of: [male, female]. The gender of the axolotl (\'male\', \'female\').", "size": "string. One of: [small, medium, large]. The size of the axolotl (\'small

In [9]:
getRandomAxolotlImage = tools['getRandomAxolotlImage']
getRandomAxolotlImage._run('{}')

{}
{'operationId': 'getRandomAxolotlImage', 'description': 'Retrieve a random axolotl image with information on the image source.', 'responses': {'200': {'description': 'Successful response', 'content': {'application/json': {'schema': {'type': 'object', 'properties': {'url': {'type': 'string', 'description': 'The URL of the axolotl image'}, 'source': {'type': 'string', 'description': 'The source of the axolotl image'}, 'description': {'type': 'string', 'description': 'A brief description of the axolotl in the image'}}}}}}}}
[]
[]
[]


'Status Code: 200. Response: {"status":"valid","standardizedAddress":{"addressLine1":"321 Maple Dr","addressLine2":"Apt 12C","city":"Houston","state":"TX","zipCode":"77002"}}'

Что если мы попробуем вызвать тул с параметрами неправильного типа

In [10]:
getRandomAxolotlImage._run('{"test": 10}')

{'test': 10}
{'operationId': 'getRandomAxolotlImage', 'description': 'Retrieve a random axolotl image with information on the image source.', 'responses': {'200': {'description': 'Successful response', 'content': {'application/json': {'schema': {'type': 'object', 'properties': {'url': {'type': 'string', 'description': 'The URL of the axolotl image'}, 'source': {'type': 'string', 'description': 'The source of the axolotl image'}, 'description': {'type': 'string', 'description': 'A brief description of the axolotl in the image'}}}}}}}}
[]
[]
[]


'Status Code: 200. Response: {"status":"valid","standardizedAddress":{"addressLine1":"321 Maple Dr","addressLine2":"Apt 12C","city":"Houston","state":"TX","zipCode":"77002"}}'

In [11]:
searchAxolotlImages = tools['searchAxolotlImages']
searchAxolotlImages._run('{"color": 10}')

{'color': 10}
{'operationId': 'searchAxolotlImages', 'description': 'Search for axolotl images based on specific criteria such as color, gender, and size.', 'parameters': [{'name': 'color', 'in': 'query', 'description': "The color of the axolotl (e.g., 'wild', 'leucistic', 'albino', etc.)", 'required': False, 'schema': {'type': 'string', 'enum': ['wild', 'leucistic', 'albino']}}, {'name': 'gender', 'in': 'query', 'description': "The gender of the axolotl ('male', 'female')", 'required': False, 'schema': {'type': 'string', 'enum': ['male', 'female']}}, {'name': 'size', 'in': 'query', 'description': "The size of the axolotl ('small', 'medium', 'large')", 'required': False, 'schema': {'type': 'string', 'enum': ['small', 'medium', 'large']}}, {'name': 'page', 'in': 'query', 'description': 'The page number for pagination purposes', 'required': False, 'schema': {'type': 'integer'}}], 'responses': {'200': {'description': 'Successful response', 'content': {'application/json': {'schema': {'type

'Parameter type error: "color", expected string, but got integer. You need to change the input and try again.\nParameter type error: "color", expected one of [\'wild\', \'leucistic\', \'albino\'], but got "10". You need to change the input and try again.'

In [12]:
function_doc = yaml.load("{'operationId': 'searchAxolotlImages', 'description': 'Search for axolotl images based on specific criteria such as color, gender, and size.', 'parameters': [{'name': 'color', 'in': 'query', 'description': \"The color of the axolotl (e.g., 'wild', 'leucistic', 'albino', etc.)\", 'required': False, 'schema': {'type': 'string', 'enum': ['wild', 'leucistic', 'albino']}}, {'name': 'gender', 'in': 'query', 'description': \"The gender of the axolotl ('male', 'female')\", 'required': False, 'schema': {'type': 'string', 'enum': ['male', 'female']}}, {'name': 'size', 'in': 'query', 'description': \"The size of the axolotl ('small', 'medium', 'large')\", 'required': False, 'schema': {'type': 'string', 'enum': ['small', 'medium', 'large']}}, {'name': 'page', 'in': 'query', 'description': 'The page number for pagination purposes', 'required': False, 'schema': {'type': 'integer'}}], 'responses': {'200': {'description': 'Successful response', 'content': {'application/json': {'schema': {'type': 'object', 'properties': {'results': {'type': 'array', 'items': {'type': 'object', 'properties': {'url': {'type': 'string', 'description': 'The URL of the axolotl image'}, 'source': {'type': 'string', 'description': 'The source of the axolotl image'}, 'description': {'type': 'string', 'description': 'A brief description of the axolotl in the image'}}}}, 'pagination': {'type': 'object', 'properties': {'current_page': {'type': 'integer', 'description': 'The current page number'}, 'total_pages': {'type': 'integer', 'description': 'The total number of pages'}, 'total_results': {'type': 'integer', 'description': 'The total number of results'}}}}}}}}}}", Loader=Loader)

In [13]:
input_params = {"color": 10}

In [14]:
function_doc['parameters']

[{'name': 'color',
  'in': 'query',
  'description': "The color of the axolotl (e.g., 'wild', 'leucistic', 'albino', etc.)",
  'required': False,
  'schema': {'type': 'string', 'enum': ['wild', 'leucistic', 'albino']}},
 {'name': 'gender',
  'in': 'query',
  'description': "The gender of the axolotl ('male', 'female')",
  'required': False,
  'schema': {'type': 'string', 'enum': ['male', 'female']}},
 {'name': 'size',
  'in': 'query',
  'description': "The size of the axolotl ('small', 'medium', 'large')",
  'required': False,
  'schema': {'type': 'string', 'enum': ['small', 'medium', 'large']}},
 {'name': 'page',
  'in': 'query',
  'description': 'The page number for pagination purposes',
  'required': False,
  'schema': {'type': 'integer'}}]

In [15]:
function_doc

{'operationId': 'searchAxolotlImages',
 'description': 'Search for axolotl images based on specific criteria such as color, gender, and size.',
 'parameters': [{'name': 'color',
   'in': 'query',
   'description': "The color of the axolotl (e.g., 'wild', 'leucistic', 'albino', etc.)",
   'required': False,
   'schema': {'type': 'string', 'enum': ['wild', 'leucistic', 'albino']}},
  {'name': 'gender',
   'in': 'query',
   'description': "The gender of the axolotl ('male', 'female')",
   'required': False,
   'schema': {'type': 'string', 'enum': ['male', 'female']}},
  {'name': 'size',
   'in': 'query',
   'description': "The size of the axolotl ('small', 'medium', 'large')",
   'required': False,
   'schema': {'type': 'string', 'enum': ['small', 'medium', 'large']}},
  {'name': 'page',
   'in': 'query',
   'description': 'The page number for pagination purposes',
   'required': False,
   'schema': {'type': 'integer'}}],
 'responses': {'200': {'description': 'Successful response',
   'co

In [16]:
'requestBody' in function_doc

False

Почему-то дефолтный парсер не ловит несоответствие типов

In [17]:
tools['searchAxolotlImages']

Tool(name='searchAxolotlImages', description='Search for axolotl images based on specific criteria such as color, gender, and size.\nParameters: {{"color": "string. One of: [wild, leucistic, albino]. The color of the axolotl (e.g., \'wild\', \'leucistic\', \'albino\', etc.).", "gender": "string. One of: [male, female]. The gender of the axolotl (\'male\', \'female\').", "size": "string. One of: [small, medium, large]. The size of the axolotl (\'small\', \'medium\', \'large\').", "page": "integer. The page number for pagination purposes."}}\nOutput: Successful response.\n - Format: application/json\n - Structure: Object{{results: Array[Object{{url, source, description}}], pagination: Object{{current_page, total_pages, total_results}}}}', args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fda8791f610>, func=<function Tool.__init__.<locals>.func at 0x7fda89757e20>, coroutine=None, max_output_len=3000)

In [18]:
type_check('color', function_doc['parameters'][0]['schema'], input_params)

[('color', 'string', 'integer'),
 ('color', "one of ['wild', 'leucistic', 'albino']", '"10"')]

In [19]:
openapi_spec

{'openapi': '3.0.0',
 'info': {'title': 'Axolotl',
  'version': '1.0.0',
  'description': 'Collection of axolotl pictures and facts'},
 'paths': {'/random': {'get': {'operationId': 'getRandomAxolotlImage',
    'description': 'Retrieve a random axolotl image with information on the image source.',
    'responses': {'200': {'description': 'Successful response',
      'content': {'application/json': {'schema': {'type': 'object',
         'properties': {'url': {'type': 'string',
           'description': 'The URL of the axolotl image'},
          'source': {'type': 'string',
           'description': 'The source of the axolotl image'},
          'description': {'type': 'string',
           'description': 'A brief description of the axolotl in the image'}}}}}}}}},
  '/search': {'get': {'operationId': 'searchAxolotlImages',
    'description': 'Search for axolotl images based on specific criteria such as color, gender, and size.',
    'parameters': [{'name': 'color',
      'in': 'query',
    

Выведем все openapi_spec

In [20]:
all_openapi_specs = []
data_simulated = json.load(open('../data/train_data.json', "r"))
for item in data_simulated:
    openapi_spec = load_openapi_spec(item["Documentation"], replace_refs=True)
    for idx, func_name in enumerate(item["Function_Projection"]):
        path, method = item["Function_Projection"][func_name]
        all_openapi_specs.append(openapi_spec["paths"][path][method])

In [21]:
len(all_openapi_specs)

2292

In [22]:
all_openapi_specs[:10]

[{'operationId': 'sendHttpRequest',
  'description': 'Send an HTTP request with the specified method, headers, and data to the Httpbin API for testing purposes.',
  'parameters': [{'name': 'method',
    'in': 'query',
    'description': 'The HTTP method to use (GET, POST, PUT, DELETE, HEAD, or PATCH)',
    'required': True,
    'schema': {'type': 'string',
     'enum': ['GET', 'POST', 'PUT', 'DELETE', 'HEAD', 'PATCH']}},
   {'name': 'url',
    'in': 'query',
    'description': 'The endpoint URL to send the request to',
    'required': True,
    'schema': {'type': 'string'}},
   {'name': 'headers',
    'in': 'query',
    'description': 'A key-value pair of headers to include in the request',
    'required': False,
    'schema': {'type': 'object'}},
   {'name': 'data',
    'in': 'query',
    'description': 'A key-value pair of data to include in the request body',
    'required': False,
    'schema': {'type': 'object'}}],
  'requestBody': {'description': 'The HTTP request body',
   'requ

In [23]:
request_body_count = 0
parameters_count = 0
for item in all_openapi_specs:
    if 'requestBody' in item:
        request_body_count += 1
        test_api_description = item
        print(item)
        print(item["requestBody"].get("content", {}).get("application/json", {}).get("schema", {}))
        print('#' * 100)
    if 'parameters' in item:
        parameters_count += 1

{'operationId': 'sendHttpRequest', 'description': 'Send an HTTP request with the specified method, headers, and data to the Httpbin API for testing purposes.', 'parameters': [{'name': 'method', 'in': 'query', 'description': 'The HTTP method to use (GET, POST, PUT, DELETE, HEAD, or PATCH)', 'required': True, 'schema': {'type': 'string', 'enum': ['GET', 'POST', 'PUT', 'DELETE', 'HEAD', 'PATCH']}}, {'name': 'url', 'in': 'query', 'description': 'The endpoint URL to send the request to', 'required': True, 'schema': {'type': 'string'}}, {'name': 'headers', 'in': 'query', 'description': 'A key-value pair of headers to include in the request', 'required': False, 'schema': {'type': 'object'}}, {'name': 'data', 'in': 'query', 'description': 'A key-value pair of data to include in the request body', 'required': False, 'schema': {'type': 'object'}}], 'requestBody': {'description': 'The HTTP request body', 'required': False, 'content': {'application/json': {'schema': {'type': 'object'}}}}, 'respons

In [24]:
request_body_count

472

In [25]:
parameters_count

1638

In [26]:
test_api_description

{'operationId': 'removeUserFromBoard',
 'description': 'Remove a user from a specific board.',
 'requestBody': {'description': 'JSON object with the following keys:',
  'required': True,
  'content': {'application/json': {'schema': {'type': 'object',
     'properties': {'boardId': {'type': 'string'},
      'userId': {'type': 'string'}}}}}},
 'responses': {'200': {'description': 'User removed from board successfully.',
   'content': {'application/json': {'schema': {'type': 'object',
      'properties': {'userId': {'type': 'string'}}}}}}}}

In [27]:
test_api_description["requestBody"].get("content", {}).get("application/json", {}).get("schema", {})

{'type': 'object',
 'properties': {'boardId': {'type': 'string'}, 'userId': {'type': 'string'}}}

Проверим несколько типов функций

In [28]:
tools

{'getRandomAxolotlImage': Tool(name='getRandomAxolotlImage', description='Retrieve a random axolotl image with information on the image source.\nParameters: {{}}\nOutput: Successful response.\n - Format: application/json\n - Structure: Object{{url, source, description}}', args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fda8791f610>, func=<function Tool.__init__.<locals>.func at 0x7fda89757d90>, coroutine=None, max_output_len=3000),
 'searchAxolotlImages': Tool(name='searchAxolotlImages', description='Search for axolotl images based on specific criteria such as color, gender, and size.\nParameters: {{"color": "string. One of: [wild, leucistic, albino]. The color of the axolotl (e.g., \'wild\', \'leucistic\', \'albino\', etc.).", "gender": "string. One of: [male, female]. The gender of the axolotl (\'male\', \'female\').", "size": "string. One of: [small, medium, large]. The size of the axolotl (\'small

In [29]:
tools['searchAxolotlImages']._run('{"color": 10}')

{'color': 10}
{'operationId': 'searchAxolotlImages', 'description': 'Search for axolotl images based on specific criteria such as color, gender, and size.', 'parameters': [{'name': 'color', 'in': 'query', 'description': "The color of the axolotl (e.g., 'wild', 'leucistic', 'albino', etc.)", 'required': False, 'schema': {'type': 'string', 'enum': ['wild', 'leucistic', 'albino']}}, {'name': 'gender', 'in': 'query', 'description': "The gender of the axolotl ('male', 'female')", 'required': False, 'schema': {'type': 'string', 'enum': ['male', 'female']}}, {'name': 'size', 'in': 'query', 'description': "The size of the axolotl ('small', 'medium', 'large')", 'required': False, 'schema': {'type': 'string', 'enum': ['small', 'medium', 'large']}}, {'name': 'page', 'in': 'query', 'description': 'The page number for pagination purposes', 'required': False, 'schema': {'type': 'integer'}}], 'responses': {'200': {'description': 'Successful response', 'content': {'application/json': {'schema': {'type

'Parameter type error: "color", expected string, but got integer. You need to change the input and try again.\nParameter type error: "color", expected one of [\'wild\', \'leucistic\', \'albino\'], but got "10". You need to change the input and try again.'

In [31]:
test_api_description['requestBody'].get("content", {}).get("application/json", {}).get("schema", {})

{'type': 'object',
 'properties': {'boardId': {'type': 'string'}, 'userId': {'type': 'string'}}}

Проверим функцию, в которой есть 

In [33]:
type_check('boardId', {'type': 'string'}, {'boardId': 10, 'type': {'test': 10}})

[('boardId', 'string', 'integer')]